In [166]:
import pandas as pd
import numpy as np

prefix = 'cards_balance_20160705'
endDate = '2016-5-5 00:00:00'

data = pd.read_csv(prefix+'.csv' ,encoding="gbk")
print len(data) # 6181

6181


In [167]:
'''
计算银行卡数量异常
规则：
    加盟日期在 60天前，发卡数量小于10 或 发卡数量大于2000的
    正式运营的
'''
cardErr = data.loc[ ( (data[u'站点加盟日期']<endDate) & ( (data[u'站点卡数']<=10) | (data[u'站点卡数']>2000) ) 
                   &(data[u'标签']==u'正式运营')
                    )
                  #,[u'站点卡数'] 
                  ]

#cardErr.describe()
print len(cardErr) # 1487
cardErr.to_csv(prefix+'cardErr'+'.csv',encoding="gbk")

342


In [168]:
'''
计算 站点余额 数量异常
规则：
    加盟日期在 60天前，站点余额 数量小于100 或 发卡数量大于 50000000 五千万 的
'''

#先看看大概
balanceErr = data.loc[ ( (data[u'站点加盟日期']<endDate) & ( (data[u'站点余额']<=100) | (data[u'站点余额']>50000000) ) 
                        &(data[u'标签']==u'正式运营')
                       )
                  ]

#cardErr.describe()
print len(balanceErr) #1074(如果算'正式运营'的话，只有51) , 其中 小于10的有1051;
balanceErr
balanceErr.to_csv(prefix+'balanceErr'+'.csv',encoding="gbk")

58


In [169]:
'''
计算 卡均余额 数量异常
规则：
    加盟日期在 60天前，卡均余额 数量小于100 或 无数据'\N'
'''

#先看看大概
# 
avgBalanceErr = data.loc[ ( (data[u'站点加盟日期']<endDate) & ( (data[u'卡均余额']<=100) | (data[u'卡均余额']=='\N') ) 
                          #&(data[u'标签']==u'正式运营')
                          )
                  #,[u'站点卡数'] 
                  ]

print len(avgBalanceErr.count()) #14 ,其中'正式运营' 只有 14
#avbBalanceErr
avgBalanceErr.to_csv(prefix+'avgBalanceErr'+'.csv',encoding="gbk")

14


In [170]:
'''
计算 户数 总人口数 数量异常
规则：
    加盟日期在 60天前，1< 总人口数/户数<8

'''

temp = data.loc[((data[u'站点加盟日期']<endDate) & ( data[u'户数']!='\N') & (data[u'总人口数']!='\N') )]

'''
户数                  object
总人口数              object
'''
temp[[u'户数',u'总人口数']] = temp[[u'户数',u'总人口数']].astype(float)
temp.dtypes
#赋值
temp['t'] = temp[u'总人口数'] / temp[u'户数']
'''
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
'''
#temp.loc[:,14]  = temp[u'总人口数'] / temp[u'户数']
temp.describe()# 先看看 temp[u'总人口数'] / temp[u'户数'] 的数值分布

familyAvgErr = temp.loc[ ( (temp[u't']<2 )| (temp[u't']>8 ))
                  ]
print "NOT 1<总人口数/户数<8：   %d " % len(familyAvgErr.index) 
familyAvgErr.to_csv(prefix+'familyAvgErr'+'.csv',encoding="gbk")#206

NOT 1<总人口数/户数<8：   206 


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [171]:
'''
计算 户数 数量异常
规则：
    加盟日期在 60天前，100<=户数<=5000
'''
ttt = data.loc[( (data[u'站点加盟日期']<endDate) 
                & ( data[u'户数']!='\N')
                &(data[u'标签']==u'正式运营')
               )]
ttt[[u'户数',u'总人口数']] = ttt[[u'户数',u'总人口数']].astype(float)

ttt.dtypes

familyErr = ttt.loc[ ( (ttt[u'站点加盟日期']<endDate) & ( ( ttt[u'户数']>5000 ) | ( ttt[u'户数']<100 ))
                      #&(ttt[u'标签']==u'正式运营')
                      )
                  #,[u'站点卡数'] 
                  ]

print "NOT 100<=户数<=5000：   %d " % len(familyErr)
familyErr 
familyErr.to_csv(prefix+'familyErr'+'.csv',encoding="gbk")

NOT 100<=户数<=5000：   89 


In [172]:
'''
计算 户数 总人口数 数量异常
规则：
    加盟日期在 60天前，400<=总人口数<=30000
'''
ppp = data.loc[( (data[u'站点加盟日期']<endDate) & (data[u'总人口数']!='\N')
               #&(data[u'标签']==u'正式运营')
               )]
ppp[[u'户数',u'总人口数']] = ppp[[u'户数',u'总人口数']].astype(float)

ppp.dtypes

populationErr = ppp.loc[( (ppp[u'总人口数']<400) | (ppp[u'总人口数']>30000)) ]

print "NOT 100<=总人口数<=5000：   %d " % len(populationErr) #119

populationErr.to_csv(prefix+'populationErr'+'.csv',encoding="gbk")

NOT 100<=总人口数<=5000：   119 


In [173]:
#data.sort_values(by=u'站点卡数')

#~企业，非正式运营，无数据，虚拟, 无数据，其他
okNode = data.loc[(data[u'站点加盟日期']<endDate)&(data[u'标签']==u'正式运营')
                   ]
entNode = data.loc[(data[u'站点加盟日期']<endDate)&(data[u'标签']==u'企业')
                   ]
notOkNode = data.loc[(data[u'站点加盟日期']<endDate)&(data[u'标签']==u'非正式运营')
                   ] 
virNode = data.loc[(data[u'站点加盟日期']<endDate)&(data[u'标签']==u'虚拟')
                   ] 
noDataNode = data.loc[(data[u'站点加盟日期']<endDate)&(data[u'标签']==u'无数据')
                   ] 
otherNode = data.loc[(data[u'站点加盟日期']<endDate)&(data[u'标签']==u'其他')
                   ] 
#okNode.to_csv(prefix+'cardErr'+'.csv',encoding="gbk")
print "全部网点数：        %d " % len(data[(data[u'站点加盟日期']<endDate)].index)
print "正式运营 网点数：   %d " % len(okNode.index) 
print "企业 网点数：       %d " % len(entNode.index)
print "非正式运营 网点数： %d " % len(notOkNode.index) 
print "虚拟 网点数：       %d " % len(virNode.index)
print "无数据 网点数:      %d " % len(noDataNode.index)
print "其他 网点数:        %d " % len(otherNode.index)
'''
全部网点数：        4859 
正式运营 网点数：   3121 
企业 网点数：       6 
非正式运营 网点数： 746 
虚拟 网点数：       2 
无数据 网点数:      981 
其他 网点数:        3 
'''

全部网点数：        4859 
正式运营 网点数：   3121 
企业 网点数：       6 
非正式运营 网点数： 746 
虚拟 网点数：       2 
无数据 网点数:      981 
其他 网点数:        3 


'\n\xe5\x85\xa8\xe9\x83\xa8\xe7\xbd\x91\xe7\x82\xb9\xe6\x95\xb0\xef\xbc\x9a        4859 \n\xe6\xad\xa3\xe5\xbc\x8f\xe8\xbf\x90\xe8\x90\xa5 \xe7\xbd\x91\xe7\x82\xb9\xe6\x95\xb0\xef\xbc\x9a   3121 \n\xe4\xbc\x81\xe4\xb8\x9a \xe7\xbd\x91\xe7\x82\xb9\xe6\x95\xb0\xef\xbc\x9a       6 \n\xe9\x9d\x9e\xe6\xad\xa3\xe5\xbc\x8f\xe8\xbf\x90\xe8\x90\xa5 \xe7\xbd\x91\xe7\x82\xb9\xe6\x95\xb0\xef\xbc\x9a 746 \n\xe8\x99\x9a\xe6\x8b\x9f \xe7\xbd\x91\xe7\x82\xb9\xe6\x95\xb0\xef\xbc\x9a       2 \n\xe6\x97\xa0\xe6\x95\xb0\xe6\x8d\xae \xe7\xbd\x91\xe7\x82\xb9\xe6\x95\xb0:      981 \n\xe5\x85\xb6\xe4\xbb\x96 \xe7\xbd\x91\xe7\x82\xb9\xe6\x95\xb0:        3 \n'